In [ ]:
# ============================================================
# AE-only Cross-Dataset Detector (Binary: Benign vs Attack)
# - Trains AE on benign-only sequences from train dataset.
# - Uses reconstruction error + threshold to detect Attack.
# - Per-day reports include overall binary and per-attack (DoS/DDoS) binary
#   metrics, each with attack-specific thresholds (PR-curve tuned).
# - Supports global or day-specific thresholds (unsupervised).
# - Overlap windows by setting stride < seq_len (e.g., stride = seq_len // 2).
# ============================================================
import os
import gc
import numpy as np
import pandas as pd



from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score, precision_recall_fscore_support,roc_auc_score,precision_recall_curve
   


from utilis.constants import DATASET_PATHS
#------------------------------------------------------------
# 1) Data loader (your function)
# ------------------------------------------------------------
from utilis.Data_loader import load_and_align_all_data
# Returns:
# (
#   all_combined_dfs,
#   all_individual_dfs_by_dataset,
#   label_encoder,
#   ALL_ENCODED_LABELS,
#   common_features,
#   broad_label_mapper,
#   broad_label_encoder
# )
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0=all, 1=info, 2=warning, 3=error

import matplotlib.pyplot as plt
import os
import json


# --- TensorFlow and Keras Imports ---
# Ensure you have tensorflow and scikit-learn installed:
# pip install tensorflow scikit-learn matplotlib
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from tensorflow.keras.models import load_model



In [32]:

# ------------------------------------------------------------
# 2) Utilities
# ------------------------------------------------------------
def build_sequences_matrix(X, y, seq_len=20, stride=None):
    """
    Build sliding windows from flat arrays.
    label = last element label in the window.
    - Non-overlapping: stride == seq_len
    - Overlapping: stride < seq_len (e.g., seq_len//2 for 50% overlap)
    """
    if stride is None:
        stride = seq_len // 2  # non-overlapping by default
    n = len(X)
    seqs = []
    labs = []
    for i in range(0, n - seq_len + 1, stride):
        seqs.append(X[i:i+seq_len])
        labs.append(y[i+seq_len-1])
    return np.asarray(seqs), np.asarray(labs)


# -------------------------------------------------------------------
# Deterministic time-ordered split (avoid temporal leakage)
# -------------------------------------------------------------------
def time_ordered_split(X, val_ratio=0.1):
    """
    Deterministic split preserving time order: first (1 - val_ratio) -> train,
    last val_ratio -> validation. No shuffling, avoids temporal leakage.
    """
    n = len(X)
    n_val = int(np.floor(val_ratio * n))
    n_train = n - n_val
    X_tr = X[:n_train]
    X_val = X[n_train:]
    return X_tr, X_val


def train_val_split(X, y, val_ratio=0.1, shuffle=True, seed=42):
    rng = np.random.default_rng(seed)
    idx = np.arange(len(y))
    if shuffle:
        rng.shuffle(idx)
    n_val = int(len(y) * val_ratio)
    val_idx = idx[:n_val]
    train_idx = idx[n_val:]
    return X[train_idx], y[train_idx], X[val_idx], y[val_idx]

def get_benign_id(broad_label_encoder):
    return broad_label_encoder.transform(["Benign"])[0]

 


In [33]:

def train_or_load_ae(
        X_train_seq, 
        seq_len, 
        n_features, 
        train_name , 
        ae_epochs=10, 
        batch_size=256, 
        verbose=1,
        latent=64,
        enc_units=(128,),
        dec_units=(128,),
        models_dir="lstmmodels",
        compile_loaded=False,  # set True if you plan to continue training loaded models
        X_val_seq=None,
        shuffle_fit=False,
        monitor="val_loss",
        patience=3,
        build_lstm_autoencoder_fn=None  # inject your builder to avoid circular imports
):
    """
        Train or load an LSTM Autoencoder (AE) and its Encoder submodel.

        - If saved models exist (matching dataset tag), load them.
        - Else, train AE on X_train_seq (or a benign-only subset you pass in),
        validate with 10% split, save AE/Encoder and history.

        Returns:
            ae (tf.keras.Model)
            encoder (tf.keras.Model)
            history (dict)  # {} if not available on load
    """
    if build_lstm_autoencoder_fn is None:
        raise ValueError("Please pass build_lstm_autoencoder_fn=build_lstm_autoencoder")
    # --- Infer dataset tag ---
    if "2017" in train_name:
        dataset_tag = "2017"
    elif "2018" in train_name:
        dataset_tag = "2018"
    else:
        dataset_tag = train_name  # fallback if other dataset names used
    
    # include key config in filenames to avoid mismatches across runs
    cfg_tag = f"sl{seq_len}_nf{n_features}_lat{latent}_enc{'-'.join(map(str, enc_units))}_dec{'-'.join(map(str, dec_units))}"
    os.makedirs(models_dir, exist_ok=True)
    ae_path   = os.path.join(models_dir, f"ae_{dataset_tag}_{cfg_tag}.h5")
    enc_path  = os.path.join(models_dir, f"encoder_{dataset_tag}_{cfg_tag}.h5")
    hist_path = os.path.join(models_dir, f"history_{dataset_tag}_{cfg_tag}.json")

    # validate inputs
    if not isinstance(X_train_seq, np.ndarray) or X_train_seq.ndim != 3:
        raise ValueError(f"X_train_seq must be a 3D ndarray (batch, seq_len, n_features); got {type(X_train_seq)} with shape {getattr(X_train_seq, 'shape', None)}")
    if X_train_seq.shape[1] != seq_len or X_train_seq.shape[2] != n_features:
        raise ValueError(f"X_train_seq shape {X_train_seq.shape} does not match seq_len={seq_len}, n_features={n_features}.")



    # attempt to load both models
    if os.path.exists(ae_path):
        print(f"[INFO] Found AE at {ae_path}. Attempting to load AE/Encoder for {dataset_tag} (cfg: {cfg_tag}) ...")
        ae = load_model(ae_path, compile=compile_loaded)
        if os.path.exists(enc_path):
            encoder = load_model(enc_path, compile=False)
        else:
            # derive encoder from AE if missing
            print(f"[WARN] Encoder file missing. Rebuilding encoder from AE (layer 'latent').")
            encoder = tf.keras.Model(ae.input, ae.get_layer("latent").output, name="LSTM_Encoder")

        # optional compile if you will continue training a loaded model
        if compile_loaded:
            ae.compile(optimizer=optimizers.Adam(1e-3), loss="mse")

        history = {}
        if os.path.exists(hist_path):
            try:
                with open(hist_path, "r") as f:
                    history = json.load(f)
            except Exception as e:
                print(f"[WARN] Failed to load history: {e}. Continuing without history.")
                history = {}
        return ae, encoder, history


    

    # else, train fresh
    print(f"[INFO] Training AE for {dataset_tag} (cfg: {cfg_tag}) ...")

    ae, encoder = build_lstm_autoencoder_fn(seq_len=seq_len,
                                         n_features=n_features,
                                         latent=latent,
                                         enc_units=enc_units,
                                         dec_units=dec_units
                                         )
    es = tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=3, restore_best_weights=True)
    ckpt = callbacks.ModelCheckpoint(
        ae_path, monitor=monitor, save_best_only=True, save_weights_only=False, verbose=1
    )

    if X_val_seq is not None:

        history_obj = ae.fit(
            X_train_seq, X_train_seq,
            validation_data=(X_val_seq, X_val_seq),
            epochs=ae_epochs,
            batch_size=batch_size,
            verbose=verbose,
            shuffle=shuffle_fit,
            callbacks=[es,ckpt]
        )
        
        # Convert to dict
        history = history_obj.history


    # Save models
     
    ae.save(ae_path)
    encoder.save(enc_path)

    try:
        with open(hist_path, "w") as f:
            json.dump(history, f)
    except Exception as e:
        print(f"[WARN] Failed to save history: {e}")

    return ae, encoder, history

In [34]:

# -------------------------------------------------------------------
# Threshold helpers (global/day and PR-curve based)
# -------------------------------------------------------------------

def choose_threshold_day(err_day, q_low=0.60, q_high=0.99, k=6.0, cap_q=0.98):
    # Unsupervised day-specific threshold from low-error subset
    if err_day.size == 0:
        return np.nan
    ben_like = err_day[err_day <= np.quantile(err_day, q_low)]
    if ben_like.size < max(100, int(0.05 * len(err_day))):
        ben_like = err_day
    med = np.median(ben_like); mad = np.median(np.abs(ben_like - med)) * 1.4826
    thr_rob = med + k * mad
    thr_q   = float(np.quantile(ben_like, q_high))
    thr_cap = float(np.quantile(err_day, cap_q))
    return min(np.median([thr_rob, thr_q]), thr_cap)

def choose_threshold_global(benign_train_errors, q=0.995, k=6.0):
    # Robust global threshold: min(q-quantile, median + k*MAD)
    if benign_train_errors.size == 0:
        return 1.0
    med = np.median(benign_train_errors)
    mad = np.median(np.abs(benign_train_errors - med)) * 1.4826
    thr_rob = med + k * mad
    thr_q   = float(np.quantile(benign_train_errors, q))
    return min(thr_rob, thr_q)


def thr_max_f1(err, y_true):
    p, r, t = precision_recall_curve(y_true, err)
    if t.size == 0:
        return None
    f1 = (2 * p[:-1] * r[:-1]) / np.clip(p[:-1] + r[:-1], 1e-12, None)
    i = int(np.nanargmax(f1))
    return float(t[i])

def thr_target_precision(err, y_true, target_p=0.60):
    p, r, t = precision_recall_curve(y_true, err)
    idx = np.where(p[:-1] >= target_p)[0]
    if idx.size == 0:
        return thr_max_f1(err, y_true)
    return float(t[idx[-1]])

def choose_day_threshold_by_pr(err_day, y_true_day, fallback_thr, mode="max_f1", target_precision=0.60):
    has_ben = (y_true_day == 0).any()
    has_att = (y_true_day == 1).any()
    if not (has_ben and has_att):
        return fallback_thr
    if mode == "max_f1":
        thr = thr_max_f1(err_day, y_true_day)
    else:
        thr = thr_target_precision(err_day, y_true_day, target_p=target_precision)
    return thr if thr is not None else fallback_thr

def choose_subset_thr(err_sub, y_true_sub, fallback_thr, mode="max_f1", target_p=0.60):
    has_ben = (y_true_sub == 0).any()
    has_att = (y_true_sub == 1).any()
    if not (has_ben and has_att):
        return fallback_thr
    if mode == "max_f1":
        thr = thr_max_f1(err_sub, y_true_sub)
    else:
        thr = thr_target_precision(err_sub, y_true_sub, target_p=target_p)
    return thr if thr is not None else fallback_thr


In [35]:
# ------------------------------------------------------------
# 3) Models: LSTM-AE  
# ------------------------------------------------------------
def build_lstm_autoencoder(seq_len, n_features, latent=64, enc_units=(128,), dec_units=(128,)):
    """
    Simple LSTM Autoencoder:
      Encoder: LSTM stacks -> latent (Dense)
      Decoder: RepeatVector -> LSTM stacks -> TimeDistributed Dense
    """
    inp = layers.Input(shape=(seq_len, n_features))
    x = inp
    for i, u in enumerate(enc_units):
        x = layers.LSTM(u, return_sequences=True, name=f"enc_lstm_{i}")(x)
        print(f"Encoder LSTM {i} output shape: (batch, {seq_len}, {u})")

    x = layers.LSTM(enc_units[-1], return_sequences=False, name="enc_final")(x)
    print(f"Final Encoder output shape: (batch, {enc_units[-1]})")

    z = layers.Dense(latent, activation='linear', name="latent")(x)
    print(f"Latent vector shape: (batch, {latent})")

    # Decoder
    x = layers.RepeatVector(seq_len, name="repeat_vector")(z)
    print(f"After RepeatVector: (batch, {seq_len}, {latent})")

    for i, u in enumerate(dec_units):
        x = layers.LSTM(u, return_sequences=True, name=f"dec_lstm_{i}")(x)
        print(f"Decoder LSTM {i} output shape: (batch, {seq_len}, {u})")

    out = layers.TimeDistributed(layers.Dense(n_features, activation='linear') , name="recon")(x)
    print(f"Reconstruction shape: (batch, {seq_len}, {n_features})")


    ae = models.Model(inp, out, name="LSTM_AE")
    ae.summary()
    # Return encoder model up to latent
    encoder = models.Model(ae.input, ae.get_layer("latent").output, name="LSTM_Encoder")
    encoder.summary()

    ae.compile(optimizer=optimizers.Adam(1e-3), loss="mse")
    return ae, encoder



In [36]:
def run_cross_ae_only(
        seq_len=20, stride=None, ae_epochs=10, batch_size=256, verbose=1, per_day=True,
                      use_day_threshold=False, q_global=0.995,qlow_day=0.60,qhigh_day=0.99, per_attack_mode="max_f1",
                      per_attack_target_precision=0.60,k=6.0,build_lstm_autoencoder_fn=None):
    (
        all_combined_dfs,
        all_individual_dfs_by_dataset,
        _,
        _,
        common_features,
        broad_label_mapper,
        broad_label_encoder
    ) = load_and_align_all_data(DATASET_PATHS)

    tf.keras.utils.set_random_seed(42)
    os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
    if stride is None:
        stride = max(1, seq_len // 2)  # overlapping by default

    scenarios = [
        ("CIC_IDS_2017", "CIC_IDS_2018"),
        ("CIC_IDS_2018", "CIC_IDS_2017"),
    ]

    results = {}

    for train_name, test_name in scenarios:
        print("\n" + "#"*80)
        print(f"### AE-only: Train on {train_name}  |  Test on {test_name}")
        print("#"*80)

        train_df = all_combined_dfs[train_name]
        test_df  = all_combined_dfs[test_name]

        X_train_flat  = train_df[common_features].fillna(0.0).to_numpy(dtype=np.float32)
        y_train_flat  = train_df["BroadLabel"].astype(int).to_numpy()
        X_test_flat   = test_df[common_features].fillna(0.0).to_numpy(dtype=np.float32)
        y_test_flat   = test_df["BroadLabel"].astype(int).to_numpy()

        benign_id = broad_label_encoder.transform(["Benign"])[0]
        dos_id    = broad_label_encoder.transform(["DoS"])[0]
        ddos_id   = broad_label_encoder.transform(["DDoS"])[0]
        allowed_labels = {benign_id, dos_id, ddos_id}

        # Filter to Benign/DoS/DDoS
        train_mask = np.isin(y_train_flat, list(allowed_labels))
        X_train_flat, y_train_flat = X_train_flat[train_mask], y_train_flat[train_mask]
        test_mask = np.isin(y_test_flat, list(allowed_labels))
        X_test_flat,  y_test_flat  = X_test_flat[test_mask], y_test_flat[test_mask]

        scaler = StandardScaler()
        X_train_flat = scaler.fit_transform(X_train_flat).astype(np.float32)
        X_test_flat  = scaler.transform(X_test_flat).astype(np.float32)

        X_train_seq, y_train_seq = build_sequences_matrix(X_train_flat, y_train_flat, seq_len=seq_len, stride=stride)
        X_test_seq,  y_test_seq  = build_sequences_matrix(X_test_flat,  y_test_flat,  seq_len=seq_len, stride=stride)
        print(f"Train sequences: {X_train_seq.shape}, Test sequences: {X_test_seq.shape}")

        # Stage 1: AE on Benign-only
        X_ae = X_train_seq[y_train_seq == benign_id]
        X_ae_tr, X_ae_val = time_ordered_split(X_ae, val_ratio=0.1)

        seq_len_eff = X_train_seq.shape[1]
        n_features  = X_train_seq.shape[2]
        ae, encoder, history = train_or_load_ae(
            X_train_seq=X_ae_tr,
            seq_len=seq_len_eff,
            n_features=n_features,
            train_name=train_name,
            ae_epochs=ae_epochs,
            batch_size=batch_size,
            verbose=verbose,
            latent=64,
            enc_units=(128,),
            dec_units=(128,),
            models_dir="lstmmodels",
           
            compile_loaded=False,
            X_val_seq=X_ae_val,
            shuffle_fit=False,
            build_lstm_autoencoder_fn=build_lstm_autoencoder
        )

        def compute_recon_error(ae_model, X):
            recon = ae_model.predict(X, verbose=0)
            return np.mean((X - recon) ** 2, axis=(1,2))

        # GLOBAL threshold from TRAIN benign errors
        benign_train_errors = compute_recon_error(ae, X_ae_tr) if X_ae_tr.size else np.array([])
        thr_global = choose_threshold_global(benign_train_errors, q=q_global, k=k)
        print(f"[INFO] Global AE threshold (q={q_global}, k={k}): {thr_global:.6f}")

        # Overall test: binary detection Attack vs Benign using AE error
        err_test = compute_recon_error(ae, X_test_seq)

        y_true_attack = np.isin(y_test_seq, [dos_id, ddos_id]).astype(int)  # 1=Attack, 0=Benign

        
        thr_used = thr_global # overall report uses global (keep simple)

        y_pred_attack = (err_test > thr_used).astype(int)

        acc = accuracy_score(y_true_attack, y_pred_attack)
        p, r, f1, _ = precision_recall_fscore_support(y_true_attack, y_pred_attack, average='binary', zero_division=0)


        try:
            has_ben = (y_true_attack == 0).any()
            has_att = (y_true_attack == 1).any()
            roc = roc_auc_score(y_true_attack, err_test) if (has_ben and has_att) else np.nan
        except Exception:
            roc = np.nan
        cm = confusion_matrix(y_true_attack, y_pred_attack, labels=[0,1])


        print(f"\n--- Overall AE-only (binary) on {test_name} ---")
        print(f"Accuracy={acc:.4f}  Precision={p:.4f}  Recall={r:.4f}  F1={f1:.4f}  ROC-AUC={roc:.4f}")
        cm = confusion_matrix(y_true_attack, y_pred_attack, labels=[0,1])
        print("Confusion matrix (rows=true [Benign,Attack], cols=pred):")
        print(cm)

        results[(train_name, test_name)] = {
            "overall_binary": {
                "accuracy": acc, "precision": p, "recall": r, "f1": f1, "roc_auc": roc,
                "threshold": thr_used, "cm": cm.tolist()
            }
        }

        # Per-day evaluation (binary)
        if per_day:
            per_day_metrics = {}
            indiv = all_individual_dfs_by_dataset[test_name]
            for day_name, df_day in indiv.items():
                if df_day.empty: 
                    continue
                    
                Xd = df_day[common_features].fillna(0.0).to_numpy(dtype=np.float32)
                yd = df_day["BroadLabel"].astype(int).to_numpy()
                Xd = scaler.transform(Xd).astype(np.float32)
                Xd_seq, yd_seq = build_sequences_matrix(Xd, yd, seq_len=seq_len, stride=stride)
                if len(yd_seq) == 0:
                    continue

                # AE errors
                err_day = compute_recon_error(ae, Xd_seq)


                # Overall day binary labels
                y_true_day_overall = np.isin(yd_seq, [dos_id, ddos_id]).astype(int)
                
                if use_day_threshold:
                    thr_day = choose_day_threshold_by_pr(
                        err_day, y_true_day_overall, fallback_thr=thr_used, mode="max_f1"
                    )
                else:
                    thr_day = thr_used

                y_pred_day_overall = (err_day > thr_day).astype(int)
                accd = accuracy_score(y_true_day_overall, y_pred_day_overall)
                pd_, rd_, f1d, _ = precision_recall_fscore_support(
                    y_true_day_overall, y_pred_day_overall, average='binary', zero_division=0
                )
                try:
                    has_ben = (y_true_day_overall == 0).any()
                    has_att = (y_true_day_overall == 1).any()
                    rocd = roc_auc_score(y_true_day_overall, err_day) if (has_ben and has_att) else np.nan
                except Exception:
                    rocd = np.nan
                cm_day = confusion_matrix(y_true_day_overall, y_pred_day_overall, labels=[0,1])

                # Per-attack (DoS / DDoS) binary reports with attack-specific thresholds
                per_attack = {}
                for attack_name, attack_id in [("DoS", dos_id), ("DDoS", ddos_id)]:
                    mask = np.isin(yd_seq, [benign_id, attack_id])
                    if not mask.any():
                        continue
                    err_sub = err_day[mask]
                    y_true_sub = (yd_seq[mask] == attack_id).astype(int)

                    thr_sub = choose_subset_thr(
                        err_sub, y_true_sub, fallback_thr=thr_day,
                        mode=per_attack_mode, target_p=per_attack_target_precision
                    )
                    y_pred_sub = (err_sub > thr_sub).astype(int)

                    has_ben_sub = (y_true_sub == 0).any()
                    has_att_sub = (y_true_sub == 1).any()

                    acc_sub = accuracy_score(y_true_sub, y_pred_sub) if (has_ben_sub or has_att_sub) else np.nan
                    prec_sub, rec_sub, f1_sub = (np.nan, np.nan, np.nan)
                    if has_ben_sub and has_att_sub:
                        prec_sub, rec_sub, f1_sub, _ = precision_recall_fscore_support(
                            y_true_sub, y_pred_sub, average='binary', zero_division=0
                        )
                    try:
                        roc_sub = roc_auc_score(y_true_sub, err_sub) if (has_ben_sub and has_att_sub) else np.nan
                    except Exception:
                        roc_sub = np.nan

                    cm_sub = confusion_matrix(y_true_sub, y_pred_sub, labels=[0,1])
  

                    per_attack[attack_name] = {
                        "threshold": thr_sub,
                        "accuracy": acc_sub,
                        "precision": prec_sub,
                        "recall": rec_sub,
                        "f1": f1_sub,
                        "roc_auc": roc_sub,
                        "cm": cm_sub.tolist(),
                        "support": {
                            "benign": int((y_true_sub == 0).sum()),
                            attack_name: int((y_true_sub == 1).sum())
                        }
                    }

                # Store per-day metrics
                per_day_metrics[day_name] = {
                    "threshold": thr_day,
                    "overall_binary": {
                        "accuracy": accd,
                        "precision": pd_,
                        "recall": rd_,
                        "f1": f1d,
                        "roc_auc": rocd,
                        "cm": cm_day.tolist(),
                        "support": {
                            "benign": int((y_true_day_overall == 0).sum()),
                            "attack": int((y_true_day_overall == 1).sum())
                        }
                    },
                    "per_attack_binary": per_attack
                }

            results[(train_name, test_name)]["per_day_binary"] = per_day_metrics
        # Cleanup
        del ae, encoder, X_train_seq, X_test_seq, X_train_flat, X_test_flat, X_ae, X_ae_tr, X_ae_val
        gc.collect()
        tf.keras.backend.clear_session()
    return results

 
# ------------------------------------------------------------
# 5) Run (AE-only) — printing per-day DoS/DDoS binary reports
# ------------------------------------------------------------
if __name__ == "__main__":
    out = run_cross_ae_only(
        seq_len=10,
        stride=10,              # try 5 for overlap
        ae_epochs=10,
        batch_size=256,
        verbose=1,
        per_day=True,
        use_day_threshold=True,
        q_global=0.995,
        qlow_day=0.60,
        qhigh_day=0.99,
        k=6.0,
        per_attack_mode="max_f1",
        per_attack_target_precision=0.60,
        build_lstm_autoencoder_fn=build_lstm_autoencoder 
    )

    # Pretty print overall AE-only (binary) metrics
    for (tr, te), res in out.items():
        ov = res["overall_binary"]
        print("\n" + "="*80)
        print(f"RESULTS  Train={tr}  Test={te}")
        print(f"Accuracy: {ov['accuracy']:.4f}  Precision: {ov['precision']:.4f}  Recall: {ov['recall']:.4f}  F1: {ov['f1']:.4f}  ROC-AUC: {ov['roc_auc']:.4f}")
        print(f"Threshold used: {ov['threshold']:.6f}")

        # Per-day AE-only (binary) metrics, including per-attack reports
        if "per_day_binary" in res:
            print("\n--- Per-Day Metrics (AE-only, binary) ---")
            for day, m in res["per_day_binary"].items():
                print(f"\nDay: {day}")
                print(f"  Threshold: {m['threshold']:.6f}")
                o = m["overall_binary"]
                print(f"  Overall  -> Acc={o['accuracy']:.4f}  Prec={o['precision']:.4f}  Rec={o['recall']:.4f}  F1={o['f1']:.4f}  ROC-AUC={o['roc_auc']:.4f}")
                print(f"              CM (rows=true [Benign,Attack], cols=pred): {o['cm']}  Support: {o['support']}")
                # Per-attack binary reports
                for atk_name, a in m.get("per_attack_binary", {}).items():
                    print(f"  {atk_name:<7} -> Acc={a['accuracy']:.4f}  Prec={a['precision']:.4f}  Rec={a['recall']:.4f}  F1={a['f1']:.4f}  ROC-AUC={a['roc_auc']:.4f}")
                    print(f"              CM (rows=true [Benign,{atk_name}], cols=pred): {a['cm']}  Support: {a['support']}")





--- Loading and Preprocessing CIC_IDS_2018 ---
[DEBUG] Now processing DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter.parquet in CIC_IDS_2018
DEBUG: Raw labels in DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter of CIC_IDS_2018 before harmonization: ['Benign', 'DDoS attacks-LOIC-HTTP']
Harmonized granular labels: {'DDoS', 'Benign'}
[DEBUG] Now processing Web1-Thursday-22-02-2018_TrafficForML_CICFlowMeter.parquet in CIC_IDS_2018
DEBUG: Raw labels in Web1-Thursday-22-02-2018_TrafficForML_CICFlowMeter of CIC_IDS_2018 before harmonization: ['Benign', 'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection']
Harmonized granular labels: {'Web Attack - Brute Force', 'Web Attack - SQL Injection', 'Web Attack - XSS', 'Benign'}
[DEBUG] Now processing Botnet-Friday-02-03-2018_TrafficForML_CICFlowMeter.parquet in CIC_IDS_2018
DEBUG: Raw labels in Botnet-Friday-02-03-2018_TrafficForML_CICFlowMeter of CIC_IDS_2018 before harmonization: ['Benign', 'Bot']
Harmonized granular labels: {'Benign', 'B

2025-09-05 23:30:55.291161: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-05 23:30:55.298083: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-05 23:30:55.302891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[INFO] Global AE threshold (q=0.995, k=6.0): 0.805462

--- Overall AE-only (binary) on CIC_IDS_2018 ---
Accuracy=0.5227  Precision=0.1838  Recall=0.6700  F1=0.2885  ROC-AUC=0.6118
Confusion matrix (rows=true [Benign,Attack], cols=pred):
[[286773 289246]
 [ 32087  65137]]

################################################################################
### AE-only: Train on CIC_IDS_2018  |  Test on CIC_IDS_2017
################################################################################
Train sequences: (673243, 10, 70), Test sequences: (246866, 10, 70)
[INFO] Found AE at lstmmodels/ae_2018_sl10_nf70_lat64_enc128_dec128.h5. Attempting to load AE/Encoder for 2018 (cfg: sl10_nf70_lat64_enc128_dec128) ...


2025-09-05 23:43:31.775097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-09-05 23:43:31.801529: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-09-05 23:43:31.807073: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

[INFO] Global AE threshold (q=0.995, k=6.0): 0.373201

--- Overall AE-only (binary) on CIC_IDS_2017 ---
Accuracy=0.6903  Precision=0.2845  Recall=0.9085  F1=0.4333  ROC-AUC=0.8892
Confusion matrix (rows=true [Benign,Attack], cols=pred):
[[141187  73510]
 [  2942  29227]]

RESULTS  Train=CIC_IDS_2017  Test=CIC_IDS_2018
Accuracy: 0.5227  Precision: 0.1838  Recall: 0.6700  F1: 0.2885  ROC-AUC: 0.6118
Threshold used: 0.805462

--- Per-Day Metrics (AE-only, binary) ---

Day: DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter
  Threshold: 0.282660
  Overall  -> Acc=0.7067  Prec=0.6706  Rec=0.9775  F1=0.7955  ROC-AUC=0.7911
              CM (rows=true [Benign,Attack], cols=pred): [[13436, 27628], [1294, 56254]]  Support: {'benign': 41064, 'attack': 57548}
  DoS     -> Acc=0.3272  Prec=nan  Rec=nan  F1=nan  ROC-AUC=nan
              CM (rows=true [Benign,DoS], cols=pred): [[13436, 27628], [0, 0]]  Support: {'benign': 41064, 'DoS': 0}
  DDoS    -> Acc=0.7067  Prec=0.6706  Rec=0.9775  F1=0.7955 